In [70]:
import subprocess
import re

In [95]:
# subprocessでコマンドを実行し、出力結果を格納
text = '藤井聡太は昨年１０月、１４歳で四段に昇段。翌１２月、デビュー戦で加藤一二三九段に勝利。藤井はそのまま連勝を続け、６月２１日に澤田真吾六段に勝ち、最多連勝記録に並んだ。'
command = 'echo "{}" | jumanpp | knp -simple -anaphora'.format(text)
result = subprocess.check_output(command, shell=True).strip()
lines = result.decode('utf-8').split('\n')

In [96]:
pat = re.compile(r'<項構造:(.*?)>')
flag = False
items = []
for line in lines:
    # 基本句かつ用言
    if line[0] == '+' and '<用言:動>' in line.split()[2]:
        flag = True
        m = pat.search(line)
        if m:
            targets = m.group(1).split(';')
            for t in targets:
                elems = t.split('/')
                item = [elems[1],elems[0],None]
                items.append(item)
    elif flag:
        # Noneの部分に原型を入れる
        for item in items:
            if item[-1] == None:
                item[-1] = line.split()[2]
        flag = False
    else:
        continue

In [97]:
from collections import defaultdict
dic = defaultdict(dict)
for item in items:
    dic[item[2]][item[1]] = item[0]
dic

defaultdict(dict,
            {'昇段': {'ガ': '藤井聡太', '時間': '昨年１０月', 'デ': '１４歳', 'ニ': '四段'},
             '勝利': {'時間': '１２月', 'デ': 'デビュー戦', 'ニ': '加藤一二三九段', 'ガ': '藤井聡太'},
             '続ける': {'ガ': '藤井聡太', '修飾': 'そのまま', 'ヲ': '連勝'},
             '勝つ': {'ニ': '澤田真吾六段', 'ガ': '藤井聡太'},
             '並ぶ': {'ニ': '最多連勝記録', 'ガ': '藤井聡太'}})

In [98]:
from graphviz import Digraph
g = Digraph(format='png')
g.attr("node", shape="square", style="filled")
sbj = 'ガ'
objs = ['ニ','ヲ']
for pred in dic:
    for obj in objs:
        if sbj in dic[pred] and obj in dic[pred]:
            g.node(dic[pred][sbj],dic[pred][sbj])
            g.node(dic[pred][sbj], shape="circle", color="pink")
            g.node(dic[pred][obj],dic[pred][obj])
            g.edge(dic[pred][sbj],dic[pred][obj],label=pred)

In [100]:
g.render('spo')

'spo.png'